# powell

In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
import scipy
from scipy.optimize import minimize, 
import warnings
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100
from scipy.optimize import minimize

importing Jupyter notebook from C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [2]:
path = "DATA/Optimizer/input.xlsx"

In [3]:
targets = ['Предел текучести',
           'Врем. сопротивление']

dir_names = ['DATA/MODELS_RF/YS RF valid', 
             'DATA/MODELS_RF/H RF valid']

models_bonds = [
    'Текучесть середина',
    'Прочность середина'
]

In [4]:
ls_columns_output = [
    '№ партии',
    '№ плавки',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'Дата термообработки',
    'ICD',
    'Примечание'
    ]

In [5]:
table_for_optimize = pd.read_excel(path, skiprows=1)

## Подготовка полученных от пользователя данных

#### Что будет вводить пользователь?  

Обязательно:
* группа прочности
* марка стали
* диаметр
* толщина стенки
* нижняя и верхняя границы допуска для предела текучести
* нижняя и верхняя границы допуска для предела прочности

Возможно:   
Режим термообработки, который нужно улучшить (он будет начальным приближением для алгоритма оптимизации):

* 1 зона по ВТР закалка
* 2 зона по ВТР закалка
* 3 зона по ВТР закалка
* Скорость прохождения трубы через спрейер, м/с
* 1 зона ВТР и уставка отпуск
* 2 зона ВТР и уставка отпуск
* 3 зона ВТР и уставка отпуск
* 4 зона ВТР и уставка отпуск
* 5 зона ВТР и уставка отпуск
* шаг балок закалочная печь, сек
* шаг балок отпускная печь, сек


Если режим термообработки не вводится, то будет подтягиваться исторический режим с минимальным шагом, в случае если шаг уже 24, режим оптимизироваться не будет (как в случае подтянутого исторического, так и в случае ручного ввода)

Сформируем списки обязательных и необязательных входных данных

In [6]:
ls_opt_need = [
    'Гр. прочн.',
    'марка стали',
    'толщина стенки',
    'диаметр'
]

bounds = [
    'Предел текучести нижняя граница',
    'Предел текучести верхняя граница',
    'Предел прочности нижняя граница',
    'Предел прочности верхняя граница'
]

ls_additional = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'Скорость прохождения трубы через спрейер, м/с',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок закалочная печь, сек',
    'шаг балок отпускная печь, сек',
    'Тип предела текучести (1186)'
]

In [7]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186)
0,К,13ХФА,72,5.5,40.0,NaN,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,К,30Г2,118,12.0,50.0,NaN,70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R95,18ХМФБ,89,9.0,66.8,88.0,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2
3,R95,Цу12,89,9.0,77.3,NaN,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2


Если верхняя границп не введена, то берем нижнюю + 20

In [8]:
def fill_up(x):
    if pd.isnull(x['Предел текучести верхняя граница']):
        x['Предел текучести верхняя граница'] = x['Предел текучести нижняя граница']+30
    if pd.isnull(x['Предел прочности верхняя граница']):
        x['Предел прочности верхняя граница'] = x['Предел прочности нижняя граница']+30
    return x
table_for_optimize = table_for_optimize.apply(fill_up, axis=1)

In [9]:
table_for_optimize['Прочность середина'] = (table_for_optimize[
    'Предел прочности нижняя граница']+table_for_optimize['Предел прочности верхняя граница'])/2.0
table_for_optimize['Текучесть середина'] = (table_for_optimize[
    'Предел текучести нижняя граница']+table_for_optimize['Предел текучести верхняя граница'])/2.0

In [10]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186),Прочность середина,Текучесть середина
0,К,13ХФА,72,5.5,40.0,70.0,60,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,55.0
1,К,30Г2,118,12.0,50.0,80.0,70,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,65.0
2,R95,18ХМФБ,89,9.0,66.8,88.0,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,77.4
3,R95,Цу12,89,9.0,77.3,107.3,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,92.3


### Подтягиваем режим, если он не введен

In [11]:
database = pd.read_csv('prepared_to_saw_gp_del_bath.csv', index_col=0)

In [12]:
database.columns

Index(['№ партии', '№ плавки', 'марка стали', 'диаметр', 'толщина стенки',
       'Гр. прочн.', '1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка', 'шаг балок закалочная печь, сек',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'шаг балок отпускная печь, сек', 'C', 'Mn', 'Si', 'P', 'S', 'Cr', 'Ni',
       'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'B', 'C-coef',
       'Параметр закалка', 'Параметр отпуск', 'Параметр отпуск новый V',
       'Величина зерна', 'Тип предела текучести (1186)', 'Дата термообработки',
       'ICD', 'Примечание', 'Предел текучести', 'Врем. сопротивление'],
      dtype='object')

In [13]:
database = database[~(database['Скорость прохождения трубы через спрейер, м/с']==1)]
database = database[~(pd.isnull(database['Скорость прохождения трубы через спрейер, м/с']))]

#### Очищаем группу прочности, файл пересохранила, теперь это нужно для самого оптимизатора только

In [14]:
replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M',
    'У':'Y',
    'Н':'H',
    'В':'B',
    'А':'A',
    'П':'n',
    'О':'O',
    'Т':'T'
}

def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

In [15]:
database.to_csv('prepared_to_saw_gp_del_bath.csv')

In [16]:
database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)
database['Примечание'] = database['Гр. прочн.'].apply(lambda x: None)

# database.to_csv('prepared_to_saw_gp_del_bath.csv')
# database

In [17]:
database[ls_columns_output+targets].to_csv('prepared_to_saw_gp_del_bath.csv')

#### Ищем ближайший режим

In [18]:
def close_value(database, col, value):
    database['diff'] = np.abs(database[col]-value)
    return database.loc[(database['diff']).argmin(),:][col]

In [19]:
def find_row_close_sort(database, row):
    for col in ls_opt_need:
        tmp = database[database[col]==row[col]]
        if tmp.shape[0]>0:
            database = tmp
        else:
            try:
                value = close_value(database, col, row[col])
                database = database[database[col]==value]
            except TypeError:
                database = database[database[col]==row[col].split('-')[0]]
                if database.shape[0]==0:
                    tmp = row
                    tmp[[col for col in row.index if col not in ls_opt_need]] = None
                    tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
                    return tmp               
    return pd.Series(database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :])

In [20]:
def find_close_sort(database, df):
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return df

In [21]:
answ = find_close_sort(database, table_for_optimize)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
answ

,1 зона ВТР и уставка отпуск,1 зона по ВТР закалка,2 зона ВТР и уставка отпуск,2 зона по ВТР закалка,3 зона ВТР и уставка отпуск,3 зона по ВТР закалка,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,Al,B,C,C-coef,Cr,Cu,ICD,Mn,Mo,N,Nb,Ni,P,S,Si,Ti,V,diff,t˚ C трубы после спреера,Величина зерна,Врем. сопротивление,Гр. прочн.,Дата термообработки,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Предел прочности верхняя граница,Предел прочности нижняя граница,Предел текучести,Предел текучести верхняя граница,Предел текучести нижняя граница,Примечание,Прочность середина,"Скорость прохождения трубы через спрейер, м/с",Текучесть середина,Тип предела текучести (1186),диаметр,марка стали,толщина стенки,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",№ партии,№ плавки
0,525.0,840.0,525.0,840.0,522.0,850.0,522.0,521.0,0.033,0.00094,0.14,0.348000,0.53,0.10,0.579200,0.48,0.007,0.00800,0.00355,0.05,0.007,0.003,0.24,0.0017,0.053,16.90,37.5,12.986261,62.500000,K,2014-12-13,838.854999,15.844163,11.137841,NaN,NaN,47.000000,NaN,NaN,NaN,NaN,0.18,NaN,0.2,88.90,13ХФА,12.20,36.0,36.0,412197,z20302
1,525.0,850.0,580.0,850.0,522.0,860.0,574.0,521.0,0.030,0.00050,0.29,0.527267,0.06,0.08,1.198637,1.29,0.005,0.00892,0.00553,0.05,0.012,0.008,0.23,0.0040,0.003,44.98,42.5,17.473343,80.000000,K,2013-12-22,847.359565,16.143345,10.032797,NaN,NaN,59.000000,NaN,NaN,NaN,NaN,0.23,NaN,0.2,73.02,30Г2,10.50,34.0,34.0,312280,v22228
2,670.0,870.0,670.0,870.0,659.0,880.0,659.0,642.0,0.031,NaN,0.16,0.523333,1.02,0.19,1.682538,0.56,0.150,0.01000,0.04000,0.14,0.010,0.007,0.23,0.0070,0.070,15.98,70.0,3.706300,86.760204,R95,2018-04-11,861.528411,18.202860,11.362842,NaN,NaN,81.632653,NaN,NaN,NaN,NaN,0.70,NaN,0.5,73.02,18ХМФБ,5.51,27.0,27.0,804140,8f1869
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.00,Цу12,9.00,NaN,NaN,Error!!! (причина:марка стали),NaN


#### Оставляем введенные режимы

Немного костыльно, если будет время нкужно будент поменять местами, сначала оставляем те, что уже введены, потом для оставшихся ищем режим.

In [23]:
try_ = table_for_optimize.copy()
try_ = try_.dropna()

In [24]:
try_

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186),Прочность середина,Текучесть середина
2,R95,18ХМФБ,89,9.0,66.8,88.0,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,77.4
3,R95,Цу12,89,9.0,77.3,107.3,70,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,92.3


In [25]:
def get_index_diff(df1,df2):
    return list(set(df1.index).difference(set(df2.index)))

In [26]:
diff = get_index_diff(answ, try_)

In [27]:
answ = pd.concat([answ.take(diff), try_])
answ = answ[ls_columns_output]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [28]:
answ = pd.concat([answ,table_for_optimize[['Прочность середина', 'Текучесть середина']]], axis=1)

In [29]:
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание,Прочность середина,Текучесть середина
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.0,0.18,37.5,525.0,525.0,522.0,522.0,521.0,36.0,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN,75.0,55.0
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.0,0.23,42.5,525.0,580.0,522.0,574.0,521.0,34.0,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN,85.0,65.0
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.0,0.20,NaN,670.0,670.0,657.0,657.0,637.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,80.0,77.4
3,NaN,NaN,Цу12,89.00,9.0,R95,840.0,840.0,840.0,28.0,0.20,NaN,670.0,670.0,657.0,657.0,637.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,80.0,92.3


###  Подтягиваем среднюю химию 

In [30]:
answ = mean_chem(answ)

Убираем строчки без химии после подтягивания

In [31]:
answ = answ[~answ['C'].isnull()]

In [32]:
answ = calc_all_features(answ)

C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "    except:\n",
C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "        pass\n",
C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

### Загрузка моделей для предсказания свойств

In [33]:
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
    try:
        scaler = StandardScaler()
        scale_data = json.load(open(dir_name+'/scaler', "r"))
        scaler.mean_ = scale_data[0]
        scaler.scale_ = scale_data[1]
    except:
        scaler=None
    return model, ls_need_col, scaler

In [34]:
models, ls_need_cols, scalers = [], [], []
for target, dir_name in zip(targets, dir_names):
    model, ls_need_col, scaler = load_model(dir_name, target)
    models.append(model)
    ls_need_cols.append(ls_need_col)
    scalers.append(scaler)

### Подготовка шагов для оптимизатора

Возникла проблема, оптимизатор не работает с двумя моделями, потому что они обучены на разном подмножестве признаков, сейчас пробую обойти это, если не получится, то нужно будет переобучить модельки.

Придумала костыль, живем дальше

In [35]:
ls_temp_z = [
    '1 зона по ВТР закалка', '2 зона по ВТР закалка',
    '3 зона по ВТР закалка'
]

ls_temp_o = [
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск'
]

# ls_s_spr = ['Скорость прохождения трубы через спрейер, м/с']

ls_shag = [
    'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'
]

ls_fit_param = ls_temp_o+ls_temp_z+ls_shag

In [36]:
X_c_down = answ.copy()
X_c_down[ls_temp_z] = X_c_down[ls_temp_z] - 30
X_c_down[ls_temp_o] = X_c_down[ls_temp_o] - 30
X_c_down[ls_shag] = 24

# X_c_down['Скорость прохождения трубы через спрейер, м/с'] = X_c_down[
#     'Скорость прохождения трубы через спрейер, м/с'].apply(lambda x: x if x>0 else 0)

X_c_up = answ.copy()
X_c_up[ls_temp_z] = X_c_up[ls_temp_z] + 30
X_c_up[ls_temp_o] = X_c_up[ls_temp_o] + 30

In [37]:
# ls_need_cols

In [38]:
def uniq(alist):    # Fastest order preserving
    set = {}
    return [set.setdefault(e,e) for e in alist if e not in set]

ls_need_union = uniq(list(sum(ls_need_cols, [])))

In [39]:
X_c_up = X_c_up[ls_need_union].dropna()
X_c_down = X_c_down[ls_need_union].dropna()

Добавлю еще один костыль в виде самостоятельной првоерки по границам.....


In [40]:
def is_in_bounds(row, bounds):
    for i in range(len(bounds)):
        if bounds[i][0]<=row[i]<=bounds[i][1]:
            pass
        else:
            return False
    return True

In [41]:
# Это нужно?
# df = pd.DataFrame(df, index=ls_need_union).T

In [42]:
tmp_try_headers = answ_n[ls_fit_param].iloc[0,:].values
pd.DataFrame(tmp_try_headers, index=ls_fit_param).T
# tmp_try_headers.shape
# ls_fit_param

NameError: name 'answ_n' is not defined

In [53]:
def model_pr(fit_params, all_params, bounds, eta):
    centr_ys = all_params['Текучесть середина']
    centr_h = all_params['Прочность середина']
    all_params = pd.concat([all_params[list(set(ls_need_union)-set(ls_fit_param))],pd.Series(fit_params, index=ls_fit_param)])
    fit_params = pd.DataFrame(fit_params, index=ls_fit_param).T
    score = 0
    
# Не нашла, что за df
#     df_ys = df[:-1]
#     df_h = np.concatenate((df[:2],df[3:-2],df[-1:]))
    
    for model_name, model, ls_need_col, scaler in zip(['ys','h'], models, ls_need_cols, scalers):
#         if pd.DataFrame(all_params).shape[0] > pd.DataFrame(all_params).shape[1]:
#             all_params = pd.DataFrame(all_params).T
#         else:
#             all_params = pd.DataFrame(all_params)
#         all_params = calc_all_features(all_params)
        if model_name=='h':
            centr=centr_h
            pred = model.predict(all_params[ls_need_col].values.reshape(1, -1))       
            tmp_score = np.abs(pred - centr)
        else:
            centr=centr_ys
            pred = model.predict(all_params[ls_need_col].values.reshape(1, -1))
            tmp_score = np.abs(pred - centr)
        if tmp_score<2:
            tmp_score=0
        if tmp_score>5:
#             tmp_score+=float('inf')
            tmp_score += 10
        score += tmp_score
        
    score += max(np.abs(all_params['шаг балок закалочная печь, сек']-24),
                 np.abs(all_params['шаг балок отпускная печь, сек']-24))*eta
#     print(score)    
    return score

In [44]:
X_c_down[ls_need_union]

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск
0,810.0,810.0,820.0,0.18,37.5,495.0,495.0,492.0,492.0,491.0,24,24,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163
1,820.0,820.0,830.0,0.23,42.5,495.0,550.0,492.0,544.0,491.0,24,24,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345


In [45]:
X_c_up[ls_need_union]

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск
0,870.0,870.0,880.0,0.18,37.5,555.0,555.0,552.0,552.0,551.0,36.0,36.0,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163
1,880.0,880.0,890.0,0.23,42.5,555.0,610.0,552.0,604.0,551.0,34.0,34.0,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345


In [46]:
answ_n = answ[ls_need_union+models_bonds].dropna()
# answ_n

In [47]:
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание,Прочность середина,Текучесть середина,Параметр отпуск новый,Параметр отпуск новый 2,T,sum_piXi,sum_qiXi
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.0,0.18,37.5,525.0,525.0,522.0,522.0,521.0,36.0,0.14000,0.48000,0.24000,0.00700,0.00300,0.53000,0.05000,0.1000,0.03300,0.05300,0.00170,0.00355,0.00700,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN,75.0,55.0,49.920500,11.618726,845.0,-2.927149,-24968.70310
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.0,0.23,42.5,525.0,580.0,522.0,574.0,521.0,34.0,0.29000,1.29000,0.23000,0.01200,0.00800,0.06000,0.05000,0.0800,0.03000,0.00300,0.00400,0.00553,0.00500,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN,85.0,65.0,49.860488,10.418611,855.0,-3.038328,-27300.77346
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.0,0.20,NaN,670.0,670.0,657.0,657.0,637.0,28.0,0.17656,0.57057,0.27012,0.00848,0.00582,0.99551,0.12524,0.1733,0.03268,0.07311,0.01323,0.04006,0.14019,0.00834,0.00000,0.533320,823.567356,18.177488,11.176230,2.502955,0.2,NaN,1.843581,NaN,80.0,77.4,49.666391,11.783254,840.0,1.471933,30276.15789


<span style="color:red"> Что-то получается, но есть проблемы... Нужно пересчитывать параметры, а не подбирать их </span>

In [48]:
answ_n

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,Текучесть середина,Прочность середина
0,840.0,840.0,850.0,0.18,37.5,525.0,525.0,522.0,522.0,521.0,36.0,36.0,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163,55.0,75.0
1,850.0,850.0,860.0,0.23,42.5,525.0,580.0,522.0,574.0,521.0,34.0,34.0,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345,65.0,85.0


In [117]:
X_a.shape[0]

2

In [54]:
%%time
answers_array = []
X_a = X_c_down[ls_need_union].copy()
X_b = X_c_up[ls_need_union].copy()

for it in range(X_a.shape[0]):
    bounds = [(i, j) for i, j in zip(X_a.loc[it,ls_fit_param], X_b.loc[it,ls_fit_param])]
    print(answ_n.iloc[it,-2],answ_n.iloc[it,-1])
    all_params = answ_n.iloc[it,:]
    fit_p = answ_n.loc[it, ls_fit_param]
    a = minimize(lambda fit_params: model_pr(fit_params, 
                                             all_params,
                                             bounds, 
                                             7),
                 fit_p, method='Powell')
    print('ВАПРОС a', it, models[1].predict(X_a.loc[it,ls_need_cols[1]].values.reshape(1, -1)),
         models[0].predict(X_a.loc[it,ls_need_cols[0]].values.reshape(1, -1)))
    print('ВАПРОС origin', it, models[1].predict(answ_n.loc[it,ls_need_cols[1]].values.reshape(1, -1)),
         models[0].predict(answ_n.loc[it,ls_need_cols[0]].values.reshape(1, -1)))
    print('ВАПРОС b', it, models[1].predict(X_b.loc[it,ls_need_cols[1]].values.reshape(1, -1)),
         models[0].predict(X_b.loc[it,ls_need_cols[0]].values.reshape(1, -1)))
    print(a)
    all_params = pd.concat([all_params[set(ls_need_union)-set(ls_fit_param)],pd.Series(a.x, index=ls_fit_param)])
    answers_array.append(all_params)
#     tmp_a_x = a['x']
#     df_h = np.concatenate((tmp_a_x[:2],tmp_a_x[3:-2],tmp_a_x[-1:]))
#     df_ys = tmp_a_x[:-1]
    ys = models[0].predict(answers_array[-1][ls_need_cols[0]].values.reshape(1, -1))
    h = models[1].predict(answers_array[-1][ls_need_cols[1]].values.reshape(1, -1))
    answers_array[-1] = pd.concat([answers_array[-1], pd.Series([ys[0],h[0]], index=['pred Прочность', 'pred Текучесть'])])
    print('АТВЕТ ', it, models[1].predict(answers_array[-1][ls_need_cols[1]].values.reshape(1, -1)),
          models[0].predict(answers_array[-1][ls_need_cols[0]].values.reshape(1, -1)))
    

55.0 75.0
ВАПРОС a 0 [66.89879796] [51.97284022]
ВАПРОС origin 0 [67.26786259] [49.87091482]
ВАПРОС b 0 [66.64452925] [48.55122904]
   direc: array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])
     fun: array(20.83017847)
 message: 'Optimization terminated successfully.'
    nfev: 829
     nit: 6
  status: 0
 success: True
       x: array([540.52757377, 534.21602159, 531.21602159, 537.52757377,
       526.02866457, 844.7096137 , 844.7096137 , 865.52757377,
        24.00000155,  23.99999989])
АТВЕТ  0 [66.77379796] [52.3960344]
65.0 85.0
ВАПРОС a 1 [86.34

In [56]:
opt_answer_end = pd.DataFrame(answers_array)[['pred Прочность', 'pred Текучесть']+list(X_c_up.columns)]
calc_all_features(opt_answer_end)

,pred Прочность,pred Текучесть,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,ICD,Параметр отпуск новый,Параметр отпуск новый 2,T,sum_piXi,sum_qiXi
0,52.396034,66.773798,844.709614,844.709614,865.527574,0.18,37.5,540.527574,534.216022,531.216022,537.527574,526.028665,24.000002,24.000000,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,844.849081,10.950701,0.348000,13.093037,15.715797,0.579200,49.515248,11.437591,855.118594,-2.927149,-24968.70310
1,69.900652,85.613142,850.857399,854.669029,865.222516,0.23,42.5,528.263264,585.222516,527.222516,579.222516,528.826603,32.572475,33.000003,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,851.485688,10.083627,0.527267,17.816002,16.240008,1.198637,49.830781,10.472335,859.945772,-3.038328,-27300.77346
